### CNN without regularization

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Using TensorFlow backend.


170500096/170498071 [==============================] - 18s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 13s 9ms/step - loss: 1.6783 - acc: 0.3925 - val_loss: 1.4981 - val_acc: 0.4460
Epoch 2/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4299 - acc: 0.4876 - val_loss: 1.2676 - val_acc: 0.5454
Epoch 3/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3062 - acc: 0.5368 - val_loss: 1.2038 - val_acc: 0.5848
Epoch 4/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2107 - acc: 0.5735 - val_loss: 1.0738 - val_acc: 0.6237
Epoch 5/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1424 - acc: 0.5967 - val_loss: 1.0057 - val_acc: 0.6538
Epoch 6/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.0870 - acc: 0.6205 - val_loss: 0.9909 - val_acc: 0.6574
Epoch 7/

1563/1563 [==============================] - 10s 6ms/step - loss: 0.5550 - acc: 0.8071 - val_loss: 0.5822 - val_acc: 0.8080
Epoch 60/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5512 - acc: 0.8087 - val_loss: 0.6079 - val_acc: 0.8006
Epoch 61/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5534 - acc: 0.8072 - val_loss: 0.6052 - val_acc: 0.8041
Epoch 62/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5469 - acc: 0.8101 - val_loss: 0.6249 - val_acc: 0.7984
Epoch 63/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5430 - acc: 0.8118 - val_loss: 0.5989 - val_acc: 0.8045
Epoch 64/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.5414 - acc: 0.8123 - val_loss: 0.6015 - val_acc: 0.8045
Epoch 65/100
1563/1563 [==============================] - 10s 7ms/step - loss: 0.5386 - acc: 0.8149 - val_loss: 0.6269 - val_acc: 0.8004
Epoch 66/100
1563/1563 [==============================

### CNN with  L2 regularization on Dense and Convolutional layers

In [5]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from numpy.random import seed
from keras import regularizers

seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_l2.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_2 = Sequential()
model_2.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_2.add(Activation('relu'))
model_2.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Flatten())
model_2.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_2.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    history_2 = model_2.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history_2 = model_2.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores_2 = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores_2[0])
print('Test accuracy:', scores_2[1])


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 11s 7ms/step - loss: 2.2331 - acc: 0.3855 - val_loss: 1.8988 - val_acc: 0.4738
Epoch 2/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.7877 - acc: 0.4802 - val_loss: 1.5657 - val_acc: 0.5426
Epoch 3/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.5915 - acc: 0.5250 - val_loss: 1.4408 - val_acc: 0.5748
Epoch 4/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4685 - acc: 0.5581 - val_loss: 1.3241 - val_acc: 0.6034
Epoch 5/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3879 - acc: 0.5831 - val_loss: 1.3025 - val_acc: 0.6154
Epoch 6/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3270 - acc: 0.5982 - val_loss: 1.2010 - val_acc: 0.6488
Epoch 7/100
1563/1563 [==============================] - 11s 7ms/step - loss

1563/1563 [==============================] - 10s 7ms/step - loss: 0.7982 - acc: 0.7817 - val_loss: 0.8139 - val_acc: 0.7837
Epoch 61/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7944 - acc: 0.7833 - val_loss: 0.8129 - val_acc: 0.7814
Epoch 62/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7933 - acc: 0.7838 - val_loss: 0.7783 - val_acc: 0.7908
Epoch 63/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7866 - acc: 0.7872 - val_loss: 0.7791 - val_acc: 0.7900
Epoch 64/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7836 - acc: 0.7874 - val_loss: 0.7941 - val_acc: 0.7842
Epoch 65/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7856 - acc: 0.7864 - val_loss: 0.7841 - val_acc: 0.7907
Epoch 66/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7838 - acc: 0.7891 - val_loss: 0.8128 - val_acc: 0.7868
Epoch 67/100
1563/1563 [==============================

In [7]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from numpy.random import seed
from keras import regularizers

seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_l2.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_3 = Sequential()
model_3.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_3.add(Activation('relu'))
model_3.add(Conv2D(32, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(64, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv2D(64, (3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Flatten())
model_3.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_3.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    history_3 = model_3.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history_3 = model_3.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores_3 = model_3.evaluate(x_test, y_test, verbose=1)
print('Test loss:', model_3[0])
print('Test accuracy:', model_3[1])


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 11s 7ms/step - loss: 2.1446 - acc: 0.3679 - val_loss: 1.8793 - val_acc: 0.4341
Epoch 2/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.7357 - acc: 0.4640 - val_loss: 1.5135 - val_acc: 0.5349
Epoch 3/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.5470 - acc: 0.5107 - val_loss: 1.4124 - val_acc: 0.5439
Epoch 4/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.4200 - acc: 0.5432 - val_loss: 1.2644 - val_acc: 0.5969
Epoch 5/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3357 - acc: 0.5676 - val_loss: 1.2338 - val_acc: 0.5915
Epoch 6/100
1563/1563 [==============================] - 10s 7ms/step - loss: 1.2787 - acc: 0.5842 - val_loss: 1.2200 - val_acc: 0.6022
Epoch 7/100
1563/1563 [==============================] - 10s 7ms/step - loss

KeyboardInterrupt: 